### MRA Project Part A

### PART A: Inferences from RFM Analysis and identified segments -> Who are your best customers? (give at least 5) -> Which customers are on the verge of churning? (give at least 5) -> Who are your lost customers? (give at least 5) -> Who are your loyal customers? (give at least 5)


### Inferences from RFM Analysis and identified segments -

### Import libraries


In [1]:
import numpy as np # linear algebra
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
RFM=pd.read_excel("Data with RFM score.xlsx");

In [6]:
cols=['CUSTOMERNAME', 'ORDERNUMBER', 'ORDERDATE', 'SALES']
RFM_df=RFM[cols]
RFM_df.head()
     

,CUSTOMERNAME,ORDERNUMBER,ORDERDATE,SALES
0,Land of Toys Inc.,10107,2018-02-24,2871.00
1,Reims Collectables,10121,2018-05-07,2765.90
2,Lyon Souveniers,10134,2018-07-01,3884.34
3,Toys4GrownUps.com,10145,2018-08-25,3746.70
4,Technics Stores Inc.,10168,2018-10-28,3479.76


In [7]:
RFM_df.shape

(2747, 4)

In [8]:
RFM_df.describe()

,ORDERNUMBER,SALES
count,2747.000000,2747.000000
mean,10259.761558,3553.047583
std,91.877521,1838.953901
min,10100.000000,482.130000
25%,10181.000000,2204.350000
50%,10264.000000,3184.800000
75%,10334.500000,4503.095000
max,10425.000000,14082.800000


In [9]:
RFM_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2747 entries, 0 to 2746
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   CUSTOMERNAME  2747 non-null   object        
 1   ORDERNUMBER   2747 non-null   int64         
 2   ORDERDATE     2747 non-null   datetime64[ns]
 3   SALES         2747 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 86.0+ KB


In [10]:
RFM_df['ORDERDATE'].max()

Timestamp('2020-05-31 00:00:00')

In [11]:
RFM_df['SALES'].max()

14082.8

### Create the RFM Table

Given dataset last order date is 31st May, 2020, which we will use to calculate recency

In [12]:
import datetime as dt
NOW = dt.datetime(2020,5,31)

In [13]:
from datetime import datetime
now = datetime.now()
     

In [14]:
RFM_table=RFM_df.groupby('CUSTOMERNAME').agg({'ORDERDATE': lambda x: (NOW - x.max()).days, # Recency
                                                'ORDERNUMBER': lambda x: len(x.unique()), # Frequency
                                                'SALES': lambda x: x.sum()})    # Monetary 

RFM_table['ORDERDATE'] = RFM_table['ORDERDATE'].astype(int)

RFM_table.rename(columns={'ORDERDATE': 'Recency', 
                         'ORDERNUMBER': 'Frequency',
                         'SALES': 'Monetary_value'}, inplace=True)

In [15]:
RFM_table.head()

,Recency,Frequency,Monetary_value
CUSTOMERNAME,,,
"AV Stores, Co.",196,3,157807.81
Alpha Cognac,64,3,70488.44
Amica Models & Co.,265,2,94117.26
"Anna's Decorations, Ltd",83,4,153996.13
Atelier graphique,188,3,24179.96


### RFM_Grouping

In [16]:
quantiles = RFM_table.quantile(q=[0.25,0.5,0.75,1.0])
quantiles

,Recency,Frequency,Monetary_value
0.25,75.0,2.0,70488.44
0.50,184.0,3.0,87489.23
0.75,230.0,3.0,120562.74
1.00,495.0,26.0,912294.11


### Converting quantiles to a dictionary, easier to use.

In [17]:
quantiles = quantiles.to_dict()
quantiles 

{'Recency': {0.25: 75.0, 0.5: 184.0, 0.75: 230.0, 1.0: 495.0},
 'Frequency': {0.25: 2.0, 0.5: 3.0, 0.75: 3.0, 1.0: 26.0},
 'Monetary_value': {0.25: 70488.44,
  0.5: 87489.23000000001,
  0.75: 120562.74,
  1.0: 912294.11}}

In [18]:
RFM_Segment = RFM_table.copy()

In [19]:
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def R_Class(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1
    
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FM_Class(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4

In [20]:
RFM_Segment['R_Quartile'] = RFM_Segment['Recency'].apply(R_Class, args=('Recency',quantiles,))
RFM_Segment['F_Quartile'] = RFM_Segment['Frequency'].apply(FM_Class, args=('Frequency',quantiles,))
RFM_Segment['M_Quartile'] = RFM_Segment['Monetary_value'].apply(FM_Class, args=('Monetary_value',quantiles,))

In [21]:
RFM_Segment['RFMClass'] = RFM_Segment.R_Quartile.map(str) \
                            + RFM_Segment.F_Quartile.map(str) \
                            + RFM_Segment.M_Quartile.map(str)

In [23]:
RFM_Segment.head(10)

,Recency,Frequency,Monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
CUSTOMERNAME,,,,,,,
"AV Stores, Co.",196,3,157807.81,2,2,4,224
Alpha Cognac,64,3,70488.44,4,2,1,421
Amica Models & Co.,265,2,94117.26,1,1,3,113
"Anna's Decorations, Ltd",83,4,153996.13,3,4,4,344
Atelier graphique,188,3,24179.96,2,2,1,221
"Australian Collectables, Ltd",22,3,64591.46,4,2,1,421
"Australian Collectors, Co.",184,5,200995.41,3,4,4,344
"Australian Gift Network, Co",119,3,59469.12,3,2,1,321
Auto Assoc. & Cie.,233,2,64834.32,1,1,1,111


In [24]:
RFM_Segment['RFMClass'].max()

'444'

### Inferences from RFM Analysis and identified segments -
### Who are your best customers? 
(BY RFMClass = 444)

In [25]:
RFM_Segment[RFM_Segment['RFMClass']=='444'].sort_values('Monetary_value', ascending=False).head(5)

,Recency,Frequency,Monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
CUSTOMERNAME,,,,,,,
Euro Shopping Channel,0,26,912294.11,4,4,4,444
Mini Gifts Distributors Ltd.,2,17,654858.06,4,4,4,444
La Rochelle Gifts,0,4,180124.90,4,4,4,444
The Sharp Gifts Warehouse,39,4,160010.27,4,4,4,444
Souveniers And Things Co.,2,4,151570.98,4,4,4,444


### Which customers are at the verge of churning? 
Customers who's recency value is low

In [26]:
RFM_Segment[RFM_Segment['R_Quartile'] <= 2 ].sort_values('Monetary_value', ascending=False).head(5)

,Recency,Frequency,Monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
CUSTOMERNAME,,,,,,,
Land of Toys Inc.,198,4,164069.44,2,4,4,244
"AV Stores, Co.",196,3,157807.81,2,2,4,224
"Saveley & Henriot, Co.",456,3,142874.25,1,2,4,124
Rovelli Gifts,201,3,137955.72,2,2,4,224
Online Diecast Creations Co.,209,3,131685.30,2,2,4,224


In [27]:
RFM_Segment['R_Quartile'].min()

1

In [28]:
RFM_Segment['RFMClass'].min()

'111'

### Who are lost customers? 
Customers who's recency, frequency as well as monetary values are low

In [29]:
RFM_Segment[RFM_Segment['RFMClass']=='111'].sort_values('Recency',ascending=False).head(5)

,Recency,Frequency,Monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
CUSTOMERNAME,,,,,,,
"Double Decker Gift Stores, Ltd",495,2,36019.04,1,1,1,111
West Coast Collectables Co.,488,2,46084.64,1,1,1,111
Signal Collectibles Ltd.,476,2,50218.51,1,1,1,111
Daedalus Designs Imports,465,2,69052.41,1,1,1,111
CAF Imports,439,2,49642.05,1,1,1,111


### Who are your loyal customers? 
Customers with high frequency value

In [30]:
RFM_Segment[RFM_Segment['F_Quartile'] >= 3 ].sort_values('Monetary_value', ascending=False).head(5)

,Recency,Frequency,Monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
CUSTOMERNAME,,,,,,,
Euro Shopping Channel,0,26,912294.11,4,4,4,444
Mini Gifts Distributors Ltd.,2,17,654858.06,4,4,4,444
"Australian Collectors, Co.",184,5,200995.41,3,4,4,344
Muscle Machine Inc,182,4,197736.94,3,4,4,344
La Rochelle Gifts,0,4,180124.90,4,4,4,444
